# Build Topic Model
The present notebook will use the data collected in section 5.1 to compute and explore a topic model. In essence, a topic model consists of two sets of probability distributions. First, the probability that a *word* belongs to a certain topic (Topic/Term probability). Second, the probability of a *topic* to appear in a document (Document/Topic probability). These probabilities are never 0, that is, each word has some probability (even if very small) to be part of each topic and each topic has some probability (even if very small) to appear in a document.

A standard way to inspect a topic model is to look at the top-ten words of a topic (the ten words with the highest probability in each particular topic). Similarly, we can pull the ten most important documents for each topic (the documents in which this topic has the highest probability).

For a more thorough analysis we may create full probability tables: a topic/term probability table and a document/topic probability table. These tables give a fuller account of the model and will be used for the visualizations (5.3). 

# TODO
Look at https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/ for ideas about hyperparameters, perplexity, convergence, etc. and https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/ for visualizations.

Also look into saving a model to disk in Gensim.

In [1]:
import numpy as np
import pandas as pd
from gensim import corpora, models, utils
import gensim
import pickle

# Read in the texts
[For test purposes one may select only the first 100 documents. Remove the hashmark (#) from the first line of the following cell if you wish to do that]

In [2]:
pickled = 'output/data_for_topic_model.p'
df = pd.read_pickle(pickled)
texts = df['lemma']

# POS-filter
The variable `posfilter` holds the last two characters of lemmatized words with allowed Part of Speech tags. If, for instance, you wish to select Verbs, Adjectives, and Nouns (in Akkadian), posfilter will be `[']n', 'aj', ']v']`. Note that one-character pos-tags need the right bracket!
The POS labels are:
* "n", #Nouns
* "v", #Verbs
* "aj", #Adjectives
* "av", #Adverbs
* "an", #Agricultural Name
* "cn", #Celestial Name
* "dn", #Divine Name
* "en", #Ethnicity Name
* "fn", #Field Name
* "gn", #Geographical Name (lands, etc.)
* "ln", #Lineage Name (ancestral clan)
* "mn", #Month Name
* "on", #Object Name
* "pn", #Personal Name
* "qn", #Quarter (of a city) Name
* "rn", #Royal Name
* "sn", #Settlement Name
* "tn", #Temple Name
* "wn", #Watercourse Name
* "yn", #Year Name
* "nu", #Numeral


In [3]:
posfilter = [']n', ']v', 'aj']
#include nouns, verbs, and adjectives, not numerals, prepositions or proper nouns
texts = [[word for word in text if word[-2:] in posfilter] for text in texts]

# Stop words

Stop words are very frequent words that are not able to distinguish between topics. This includes, for instance, prepositions - but those can also be filtered out by the POS filter. The following nouns and verbs are too frequent to contribute to the analysis. Note that this list of stop words was assembled for the SAAo corpus - another corpus may require a different list, or none at all. In a cell further below the dictionary is built - leaving out words that appear in more than 80 percent of the documents (or whatever the 'no_above' parameter is set too) making the use of a stop word list mostyly unnecessary. The only advantage of an explicit list of stop words is that it makes it possible to filter out documents or text fragments that remain with too few words to be meaningful.

The 'stoplist' cell can be omitted entirely or adapted to your purposes.

In [4]:
stoplist = [
'šarru[king]n',
'bēlu[lord]n',
'libbu[interior]n',
'muhhu[skull]n',
'ardu[slave]n',
'šulmu[completeness]n',
'šapāru[send]v',
'alāku[go]v',
'qabû[say]v',
'pānu[front]n',
'māru[son]n',
'bītu[house]n',
'epēšu[do]v',
'wabālu[bring]v',
'šakānu[put]v',
'amāru[see]v',
'bašû[exist]v',
'našû[lift]v',
'izuzzu[stand]v',
'ūmu[day]n',
'ṭābu[good]aj',
'mādu[many]aj',
'nadānu[give]v',
'tadānu[give]v',
'ṣehru[small]aj',
'mimmû[all]n',
'gimru[totality]n',
'gabbu[totality]n',
'šâlu[ask]v',
'šemû[hear]v',
'ūmu[day]n',
'awātu[word]n',
'erēbu[enter]v'
]
texts = [[word for word in text if word not in stoplist] for text in texts]


# Filter out texts that have too few words left
Identify texts that have at least 10 lemmas left and use that as a mask to filter  the list `texts` as well as the dataframe `df`. 

In [5]:
bo = [len(text)>9 for text in texts]
df = df[bo]
texts = [texts[i] for i in range(0, len(texts)) if bo[i]]

How many documents did we start with, and how many do we have left?

In [6]:
len(bo), len(df)

(4956, 2997)

# Dictionary
create the gensim Dictionary and filter for words that are too common or too rare (no_above may be set too low here).

In [7]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)
dictionary.save('output/ldadict')
## CHECK - is this done correctly?

In [8]:
corpus = [dictionary.doc2bow(doc) for doc in texts]
corpora.MmCorpus.serialize('output/ldacorpus', corpus)

# Compute the Model

Set the seed, indicate the number of topics (default set to 10) and run the model.

The visualization (section 5.3) will fail if the number of topics is higher than 25. 

In [9]:
ntopics = int(input("Number of topics: ") or 10)
if ntopics > 25:
    ntopics = 25

Number of topics:  


In [10]:
seed = 15
np.random.seed(seed)
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel
# Running and Training LDA model on the document term matrix.
ldamodel = Lda(corpus, num_topics=ntopics, id2word = dictionary, passes=150)

List the top 10 words and their probabilities in all topics. Note: the topic numbers here are not the ones used in the visualizations in 5.3! (The topics are the same, but not their numbers).

Save the model

In [11]:
saved = 'output/ldasaved'
ldamodel.save(saved)

In [12]:
ldamodel.show_topics(ntopics, formatted = False)

[(0,
  [('qû[unit]n', 0.053382657),
   ('karānu[vine]n', 0.031626757),
   ('šikaru[beer]n', 0.030481339),
   ('šamnu[oil]n', 0.02594168),
   ('kusāpu[bread]n', 0.025353106),
   ('zamāru[sing]v', 0.021011204),
   ('dišpu[honey]n', 0.018200744),
   ('naqû[pour-(a-libation)]v', 0.017905407),
   ('immeru[sheep]n', 0.017024096),
   ('kaptukkû[two-sūtu-container]n', 0.013798216)]),
 (1,
  [('rabû[big]aj', 0.02772635),
   ('abu[father]n', 0.021949498),
   ('ilu[god]n', 0.021935958),
   ('ahu[brother]n', 0.020562295),
   ('adû[(treaty-)oath]n', 0.015737671),
   ('ridûtu[appropriation]n', 0.01515395),
   ('šarrūtu[kingship]n', 0.012755803),
   ('ṣabātu[seize]v', 0.012666813),
   ('ṭuppu[tablet]n', 0.012434676),
   ('antalû[eclipse]n', 0.0123058995)]),
 (2,
  [('šību[witness]n', 0.15905823),
   ('manû[unit]n', 0.046707977),
   ('ṣarpu[silver]n', 0.036156394),
   ('eqlu[field]n', 0.03523169),
   ('dīnu[legal-decision]n', 0.026832364),
   ('dabābu[speak]v', 0.025840912),
   ('imēru[unit]n', 0.0227

# Document/Topic Probability
The function `get_document_topics()` will list the probability of the topics in a single document. In order to get all the topics set the argument `minimum_probability` to zero. 

In [13]:
ldamodel.get_document_topics(corpus[1], minimum_probability=0)

[(0, 0.0038526477),
 (1, 0.2544701),
 (2, 0.0038511516),
 (3, 0.003851311),
 (4, 0.2945981),
 (5, 0.0038508724),
 (6, 0.0038529793),
 (7, 0.0038508677),
 (8, 0.0038534175),
 (9, 0.42396858)]

# Create Document/Topic Probability Table
A Document/Topiuc probability table is a table (DataFrame), where each row represents a document and each column a topuic. Each cell has the probability of a particular topic in a particular document. The sum of each row is 1 (probability distribution).

In order to create a full Document/Topic probability table we iterate over the entire corpus with the `get_document_topics()` function. This creates a list of lists (`list_of_doctopics`) where each list represents the probability of each topic in a document. The probability is represented in a tuple (topic_number, probability). The `list_of_probabilities` preserves only the probabilities. This list of lists is transformed into a DataFrame, whith as index the index of the original DataFrame with the tokenized data.  

In [14]:
list_of_doctopics = [ldamodel.get_document_topics(corpus[i], minimum_probability=0) for i in range(len(corpus))]
list_of_probabilities = [[probability for label,probability in distribution] for distribution in list_of_doctopics]
d_t_df = pd.DataFrame(list_of_probabilities)
d_t_df = d_t_df.set_index(df.index)
d_t_df.head()

,0,1,2,3,4,5,6,7,8,9
P224378,0.009093,0.363734,0.009092,0.009092,0.235343,0.009093,0.009095,0.009093,0.009093,0.337271
P224382,0.003853,0.254477,0.003851,0.003851,0.294594,0.003851,0.003853,0.003851,0.003853,0.423965
P224383,0.003334,0.059656,0.003335,0.242550,0.674452,0.003334,0.003335,0.003335,0.003334,0.003334
P224386,0.006667,0.006670,0.006668,0.151204,0.795450,0.006668,0.006667,0.006668,0.006669,0.006669
P224388,0.006670,0.006671,0.006671,0.006671,0.865835,0.006671,0.006671,0.006671,0.080799,0.006671


We can use the above table to find the ten highest scoring documents per topic with the pandas function 'nlargest'. First add the 'designation' as a separate column to the table.

In [15]:
d_t_df_w_desig = pd.merge(df['designation'], d_t_df, left_index=True, right_index=True)
d_t_df_w_desig

,designation,0,1,2,3,4,5,6,7,8,9
P224378,Take Over the Kingship!,0.009093,0.363734,0.009092,0.009092,0.235343,0.009093,0.009095,0.009093,0.009093,0.337271
P224382,Family Affairs,0.003853,0.254477,0.003851,0.003851,0.294594,0.003851,0.003853,0.003851,0.003853,0.423965
P224383,Šubrian King Protecting Deserters,0.003334,0.059656,0.003335,0.242550,0.674452,0.003334,0.003335,0.003335,0.003334,0.003334
P224386,Specialists Reviving the Land,0.006667,0.006670,0.006668,0.151204,0.795450,0.006668,0.006667,0.006668,0.006669,0.006669
P224388,Elamite King and the Men of Mukin-zeri,0.006670,0.006671,0.006671,0.006671,0.865835,0.006671,0.006671,0.006671,0.080799,0.006671
...,...,...,...,...,...,...,...,...,...,...,...
X900011,Reclaiming Runaway Servants,0.002942,0.002943,0.227756,0.402893,0.348756,0.002942,0.002942,0.002942,0.002942,0.002942
X900012,Receiving Tribute Horses; Constructing Kar-Šar...,0.002084,0.002084,0.002084,0.559244,0.424084,0.002084,0.002084,0.002084,0.002084,0.002084
X900013,Midas of Phrygia Seeks Detente,0.001164,0.001164,0.014715,0.001164,0.975970,0.001164,0.001164,0.001164,0.001164,0.001165
X900014,Report on a Festival and on Bull Colossi,0.037467,0.002274,0.002274,0.563164,0.002274,0.002274,0.002274,0.252662,0.002274,0.133063


The following code goes through the (numbered) columns of the table which hold the probabilities of each of the topics (the columns) in each of the documents (the rows). The highest ten probabilities are selected, together with a brief descriptipon of the text (designation). 

In [16]:
doctop = []
for i in range(ntopics):
    t = d_t_df_w_desig.nlargest(10, i)[['designation', i]]
    t['topic'] = i
    t = t.rename(columns = {i :'probability'})
    doctop.append(t)
doctop_df = pd.concat(doctop, axis=0)
doctop_df

,designation,probability,topic
P336282,Unplaced Fragment of the Text of No. 69,0.976922,0
P335833,"Aššur Temple Offerings, Day 10",0.974282,0
P335824,"Aššur Temple Offerings, Day 5",0.957137,0
P425166,Fragment of a Ritual for Singer,0.952628,0
P335891,Record of Mixed Food Offerings,0.949989,0
...,...,...,...
P336453,Full Moon on 14th Day,0.974998,9
P336449,Full Moon on 14th Day,0.974996,9
P336458,Full Moon on 14th Day,0.971872,9
P237917,Full Moon on 14th Day,0.970966,9


# Renumber Topics
Rename the topics (columns) to start with 1, in accordance with the pyLDAvis visualization.

The LDAvis package, used in notebook 5.3, was originally written for statistical programming language R; pyLDAvis is a python wrapper. In R indexing starts with 1, rather than 0 (as is the case in Python). In order to prevent confusion, we will rename all topics according to the pyLDAvis convention.

In [17]:
topics = [i+1 for i in range(ntopics)]
d_t_df.columns = topics
d_t_df

,1,2,3,4,5,6,7,8,9,10
P224378,0.009093,0.363734,0.009092,0.009092,0.235343,0.009093,0.009095,0.009093,0.009093,0.337271
P224382,0.003853,0.254477,0.003851,0.003851,0.294594,0.003851,0.003853,0.003851,0.003853,0.423965
P224383,0.003334,0.059656,0.003335,0.242550,0.674452,0.003334,0.003335,0.003335,0.003334,0.003334
P224386,0.006667,0.006670,0.006668,0.151204,0.795450,0.006668,0.006667,0.006668,0.006669,0.006669
P224388,0.006670,0.006671,0.006671,0.006671,0.865835,0.006671,0.006671,0.006671,0.080799,0.006671
...,...,...,...,...,...,...,...,...,...,...
X900011,0.002942,0.002943,0.227756,0.402893,0.348756,0.002942,0.002942,0.002942,0.002942,0.002942
X900012,0.002084,0.002084,0.002084,0.559244,0.424084,0.002084,0.002084,0.002084,0.002084,0.002084
X900013,0.001164,0.001164,0.014715,0.001164,0.975970,0.001164,0.001164,0.001164,0.001164,0.001165
X900014,0.037467,0.002274,0.002274,0.563164,0.002274,0.002274,0.002274,0.252662,0.002274,0.133063


# Create Topic / Term table
This is a table with N rows (the number of topics) and M columns (the number of individual terms in the Dictionary). The table indicates the probability of each term in each topic.

In [18]:
topic_term = ldamodel.show_topics(ntopics, formatted=False, num_words=len(dictionary))

The object `topic_term` is a list of tuples in the format (index, \[list\]). The index is the topic number, the list is again a list of tuples in the form `(lemma, probability)` in descending order of probability. This means that the lemmas are in a different order in each of the lists.

The command `pd.DataFrame(topic_term[i][1])` creates a DataFrame with two columns for the topic `i`. The first column (column 0) has the lemmas, the second column the probabilities. By using the command `set_index(0)` the lemmas become the index (rather than a column) and this index can be used to concatenate the DataFrames (one for each topic) with the `sort = True` option. 

In [19]:
topic_term_list = [pd.DataFrame(topic_term[i][1]).set_index(0) for i in range(ntopics)]
t_t_df_ = pd.concat(topic_term_list, axis=1, ignore_index=True, sort=True)
t_t_df_.head()

,0,1,2,3,4,5,6,7,8,9
aban-bāšti[(a-stone)]n,0.000011,0.000009,0.000004,0.000198,0.000006,0.000007,0.000017,0.000026,0.000011,0.000008
aban-lamassi[(a-precious-stone)]n,0.000011,0.000009,0.000185,0.000012,0.000006,0.000007,0.000017,0.000026,0.000011,0.000008
aban-râmi['love'-stone]n,0.000011,0.000009,0.000004,0.000198,0.000006,0.000007,0.000017,0.000026,0.000011,0.000008
abati[(meaning-unknown)]n,0.000011,0.000009,0.000004,0.000012,0.000006,0.000007,0.000017,0.000211,0.000011,0.000008
abašmû[(a-stone)]n,0.000011,0.000009,0.000004,0.000198,0.000006,0.000007,0.000017,0.000026,0.000011,0.000008


Rename the columns to start with 1, and Transpose to Topic/Term matrix.

In [20]:
t_t_df_.columns = topics
t_t_df = t_t_df_.T
t_t_df

,aban-bāšti[(a-stone)]n,aban-lamassi[(a-precious-stone)]n,aban-râmi['love'-stone]n,abati[(meaning-unknown)]n,abašmû[(a-stone)]n,abbušu[(meaning-unknown)]n,abbūtu[fatherhood]n,abiktu[defeat]n,abku[captive]n,ablu[brought]aj,...,ṭēmūtu[of-order]n,ṭīdu[clay]n,ṭīmu[yarn]n,ṭīpu[addition]n,ṭīru[impression]n,ṭūbtu[peace]n,ṭūbu[goodness]n,ṭūbātu[happiness]n,ṭūdu[way]n,ṭūru[opopanax]n
1,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,...,0.000011,0.005888,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000369,0.000011
2,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.001295,0.000858,0.000009,0.000009,...,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009
3,0.000004,0.000185,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,...,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004
4,0.000198,0.000012,0.000198,0.000012,0.000198,0.000012,0.000012,0.000012,0.000012,0.000012,...,0.000267,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012
5,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000155,0.000006,0.000006,...,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000884,0.000006,0.000006,0.000006
6,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000113,0.000007,0.000007,...,0.000007,0.000007,0.000007,0.000007,0.000057,0.000007,0.000482,0.000706,0.000007,0.000007
7,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,...,0.000017,0.000017,0.000017,0.000279,0.000017,0.000017,0.000483,0.000017,0.000017,0.000017
8,0.000026,0.000026,0.000026,0.000211,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,...,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026
9,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000092,...,0.000011,0.000011,0.000092,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011
10,0.000008,0.000008,0.000008,0.000008,0.000008,0.000061,0.000699,0.000008,0.000170,0.000008,...,0.000008,0.000008,0.000008,0.000008,0.000008,0.000126,0.011780,0.000008,0.000008,0.000126


In [21]:
#just checking
t_t_df['ēkallu[palace]n']

1     0.000011
2     0.006446
3     0.004353
4     0.011237
5     0.011080
6     0.000386
7     0.001240
8     0.036904
9     0.002558
10    0.001400
Name: ēkallu[palace]n, dtype: float64

# Export Data

In [22]:
topic_model = {'dictionary': dictionary,
                  'corpus' : corpus,
                  'ldamodel' : ldamodel,
                  't_t_df' : t_t_df,
                  'd_t_df' : d_t_df,
               'df' : df,
              'ntopics' : ntopics,
              'texts' : texts}

In [23]:
with open('output/topic_model.p', 'wb') as w:
    pickle.dump(topic_model, w)